In [ ]:
import sys
from tqdm import tqdm

import torch
import torchvision
from torch.nn.utils import vector_to_parameters, parameters_to_vector

import nnj
from pytorch_laplace.hessian.contrastive import ArccosHessianCalculator
from pytorch_laplace.laplace.diag import DiagLaplace

sys.path.append("../../")
from plotting_fun import plot_latent, plot_reconstruction_with_latent, plot_std, plot_fancy_latent, plot_attention, plot_training

torch.manual_seed(0)

In [ ]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))


device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

batch_size = 256

dataset1 = datasets.MNIST("../../../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../../../data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(
    dataset1, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
model = nnj.Sequential(
    nnj.Conv2d(1, 32, 3, 1),
    nnj.Tanh(),
    nnj.Conv2d(32, 64, 3, 1),
    nnj.Tanh(),
    nnj.MaxPool2d(2),
    nnj.Flatten(),
    nnj.Linear(9216, 128),
    nnj.L2Norm(),
)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 1


### pytorch-metric-learning stuff ###
distance = distances.LpDistance()
loss_func = losses.ContrastiveLoss(
    pos_margin=0, neg_margin=0.8, distance=distance
)

mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###


for epoch in range(1, num_epochs + 1):
    train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

# Laplace posthoc

In [ ]:
# load model parameters


hessian_calculator = ArccosHessianCalculator(
    wrt="weight",
    shape="diagonal",
    speed="half",
    method="fix",
)

# if arccos, then remove normalization layer from model
model = nnj.convert_to_nnj(model[:-1])

laplace = DiagLaplace()
_prior_prec = 0.01 # weight of the l2 regularizer
_prior_prec_multiplier = 10000 #prior optimization (made a posteriori by the Laplace Redux guys)

In [ ]:
mean = parameters_to_vector(model.parameters())
prior_hessian = _prior_prec_multiplier *_prior_prec * torch.ones_like(mean)
hessian = 0
for batch in train_loader:
    x, y = batch

    embeddings = model(x)
    tuple_indices = mining_func(embeddings, y)
    hessian += hessian_calculator.compute_hessian(x, model)

precision = prior_hessian + hessian
std_deviation = 1.0 / precision.sqrt()

plot_attention(std_deviation)